# 03. Artificial Neural Networks with TF

<p style="text-align: right;">
rayleigh@dplus.company<br>
blackdew7@gmail.com<br>
Your name :
</p>

### Intro to TensorFlow 시간!<br>

#### 선행지식
1. 모델링을 한다는 것에 대한 이해.
2. Logistic Regression, Linear Regression
3. Activation function 과 레이어

#### 실습목표
1. ANN 의 구조를 Graph로 그려낼 수 있다.
2. 그려낸 Graph를 텐서플로우를 이용해 코딩할 수 있다.
3. ANN의 마지막 레이어에 따라서 Regression도 Classification도 가능함을 알고 코딩할 수 있다.

#### 사용데이터.

01. Multivariate Regression : http://archive.ics.uci.edu/ml/datasets/Bike+Sharing+Dataset
02. Multinomial Classification : http://yann.lecun.com/exdb/mnist/

# Regression with Artificial Neural Network

실습 01의 Bike-Sharing Data를 기억하지요?

단순히 Linear Regression을 사용하기에는 이런 저런 고민들이 됩니다.

1. 변수가 저렇게 많으면 어차피 직관적인 해석은 안된다.
2. 저 중 컨트롤 가능한 변수는 없었다.
3. 어쨌든 인풋 데이터에 따라 아웃풋 데이터가 잘 나오면 된다.

바야흐로, Artificial Neural Network가 필요한 때입니다.

### Warm up before deep playing

### 00. 라이브러리 불러오기

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import time

### 01. 저번에 했었던 모든 전처리들

In [0]:
# 데이터 불러오기
rides = pd.read_csv('https://raw.githubusercontent.com/blackdew/dplus-tensorflow/master/csv/Bike-Sharing-Hour.csv')

# 더미
dummy_fields = ['season', 'weathersit', 'mnth', 'hr', 'weekday']
for each in dummy_fields:
    dummies = pd.get_dummies(rides[each], prefix=each, drop_first=False)
    rides = pd.concat([rides, dummies], axis=1)

# 드롭
fields_to_drop = ['instant', 'dteday', 'season', 'weathersit', 'mnth', 'hr', 'weekday']
data = rides.drop(fields_to_drop, axis=1)

# 노말리이제이션(Scailing)
quant_features = ['casual', 'registered', 'cnt', 'temp', 'atemp', 'hum', 'windspeed']

# Store scalings in a dictionary so we can convert back later
scaled_features = {}
for each in quant_features:
    mean, std = data[each].mean(), data[each].std()
    scaled_features[each] = [mean, std]
    data.loc[:, each] = (data[each] - mean) / std
    
# 트레이닝 / 테스트 셋
test_data, train_data = data[-60*24:], data[:-60*24]

target_fields = ['cnt', 'casual', 'registered']
test_features, test_targets = test_data.drop(target_fields, axis=1), test_data[target_fields]
train_features, train_targets = train_data.drop(target_fields, axis=1), train_data[target_fields]

test_X, test_Y = test_features.values, test_targets.values
train_X, train_Y = train_features.values, train_targets.values

print(train_X.shape, train_Y.shape)


### 02. 손으로 그려보자 ANN

일단은 같이 해봅시다!<br>
맨 마지막에, 아무것도 안주고 여러분이 해야만 하는 부분이 있습니다!

#### 조건 :
* Hidden Layer는 3개만 쓰자.
* Hidden Layer의 노드 수는 16, 8, 4 개로 한다.
* Shape를 실수하지 말자.

### 03. Make a Graph with TF

01.Linear Regression with TF 의 코드와 직접 비교해보셔도 좋습니다

X: (15939, 58)
Y: (15939, 3)

W1: (58, 32)
B1: (32)

Wo: (32, 3)
Bo: (3)



In [0]:
'''
print(train_X.shape, train_Y.shape)
(15435, 58),   (15435, 3)
'''

# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01

# Input Layer, Real Y
X = tf.placeholder(dtype=tf.float32, shape=(None, 58))
Y = tf.placeholder(dtype=tf.float32, shape=(None, 3))

# 1st Hidden Layer
W1 = tf.Variable(tf.random_normal([58, 32]))
b1 = tf.Variable(tf.random_normal([32]))
sum1 = tf.add(tf.matmul(X, W1), b1)
hidden_output1 = tf.nn.relu(sum1)

# 2nd Hidden Layer
W2 = tf.Variable(tf.random_normal([32, 16]))
b2 = tf.Variable(tf.random_normal([16]))
sum2 = tf.add(tf.matmul(hidden_output1, W2), b2)
hidden_output2 = tf.nn.relu(sum2)

# last Hidden Layer
W3 = tf.Variable(tf.random_normal([16, 16]))
b3 = tf.Variable(tf.random_normal([16]))
sum3 = tf.add(tf.matmul(hidden_output2, W3), b3)
hidden_output3 = tf.nn.relu(sum3)

# Output Layer
Wo = tf.Variable(tf.random_normal([16, 3]))
bo = tf.Variable(tf.random_normal([3]))
Y_pred = tf.add(tf.matmul(hidden_output3, Wo), bo)

# Cost(loss) function & Optimizer
cost = tf.losses.mean_squared_error(Y, Y_pred)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


### 04. Train! Session!

In [0]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())

start = time.clock()
for i in range(5000):
    # 학습 진행
    sess.run(optimizer, feed_dict={X: train_X, Y: train_Y})
    
    if (i % 100 == 0):
        # 학습 상황 디스플레이
        tr_l = sess.run(cost, feed_dict={X: train_X, Y: train_Y})
        print("{}th training---- training loss: {:.5f}".format(i, tr_l))
        
print("Training Complete")
end = time.clock()

print("Total elapsed time: {}".format(end - start))
print("Test loss: {:.5f}".format(sess.run(cost, feed_dict={X: test_X, Y: test_Y})))

# 결과값 예측
Y_predicted = sess.run(Y_pred, feed_dict={X: test_X, Y: test_Y})

sess.close()

### Check!

In [0]:
# target_fields = ['cnt', 'casual', 'registered']
column = 'registered'
cindex = target_fields.index(column)
rows0, rows1 = 1000, 1100

fig, ax = plt.subplots(figsize=(10, 5))

# unscailing - 예측값도 Scaling 되어 있으므로 그것을 원래대로 되돌려주는 과정
mean, std = scaled_features[column]
predictions = Y_predicted[rows0:rows1, cindex] * std + mean
REAL = test_Y[rows0:rows1, cindex] * std + mean

ax.plot(predictions, label='Prediction')
ax.plot(REAL, label='Real')
ax.set_xlim(right=len(Y_predicted[rows0:rows1, cindex]))
ax.legend()

dates = pd.to_datetime(rides.iloc[test_data[rows0:rows1].index]['dteday'])
dates = dates.apply(lambda d: d.strftime('%b %d'))
ax.set_xticks(np.arange(len(dates))[12::24])
_ = ax.set_xticklabels(dates[12::24], rotation=45)

In [0]:
tf.reset_default_graph()
%reset

# Classification with Artificial Neural Network

실습 02의 MNIST Data를 기억하지요?

1. 변수가 저렇게 많으면 어차피 직관적인 해석은 안된다.
2. 저 중 컨트롤 가능한 변수는 없었다.
3. 어쨌든 인풋 데이터에 따라 아웃풋 데이터가 잘 나오면 된다.
4. 원리가 어쨌든 정확성이 높아야 한다.


### 외칩시다. NEURAL NETWORK!

### 00. 라이브러리 불러오기

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time


### 01. 저번에 했었던 모든 전처리들

In [0]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
print(tf.convert_to_tensor(mnist.train.images).get_shape())


In [0]:
mnist.test.labels.tolist()[0]

In [0]:
'''
mnist 시각화 함수
'''
def mnist_plot(i):
    pixels = mnist.test.images[i].reshape((28, 28))
    a = mnist.test.labels.tolist()
    plt.title('Label : {}'.format(a[i]))
    plt.imshow(pixels, cmap='gray')
    return plt.show()

In [0]:
'''
0 ~ 9999 사이의 숫자를 넣어서 확인해보자.
'''
mnist_plot(5)

In [0]:
'''
트레이닝 데이터로 활용할 55000개 이미지, 테스트용 10000개 이미지
28*28 사이즈의 이미지가 그냥 주욱 784칸 짜리 어레이에 담겨있다.
레이블은 one-hot encoding을 해주어야 한다. 
'''
print(mnist.train.images.shape, mnist.train.labels.shape)
print(mnist.test.images.shape, mnist.test.labels.shape)

mnist.train.labels_onehot = np.eye(10)[mnist.train.labels]
mnist.test.labels_onehot = np.eye(10)[mnist.test.labels]

print(mnist.train.images.shape, mnist.train.labels_onehot.shape)
print(mnist.test.images.shape, mnist.test.labels_onehot.shape)

In [0]:
mnist.train.labels[0]

### 02. 손으로 그려보자 ANN

일단은 같이 해봅시다!<br>
맨 마지막에, 아무것도 안주고 여러분이 해야만 하는 부분이 있습니다!

#### 조건 :
* Hidden Layer는 4개만 쓰자.
* Hidden Layer의 노드 수는 256, 128, 64, 64 개로 한다.
* Shape를 실수하지 말자.
* 인풋의 Shape = [ None, 28\*28 ], 아웃풋의 Shape = [ None, 10 ] 기억하자

### 03. Make a Graph with TF

02.Logistic Regression with TF 의 코드와 직접 비교해보셔도 좋습니다

### 02. 텐서서플로우로 그래프를 만들자!

In [0]:
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01

# Input Layer, Real Y
X = tf.placeholder(dtype=tf.float32, shape=[None, 784])
Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

# 1st Hidden Layer
W1 = tf.Variable(tf.random_normal([784, 256]))
b1 = tf.Variable(tf.random_normal([256]))
sum1 = tf.add(tf.matmul(X, W1), b1)
hidden_output1 = tf.nn.relu(sum1)

# 2nd Hidden Layer
W2 = tf.Variable(tf.random_normal([256, 256]))
b2 = tf.Variable(tf.random_normal([256]))
sum2 = tf.add(tf.matmul(hidden_output1, W2), b2)
hidden_output2 = tf.nn.relu(sum2)

# 3rd Hidden Layer
W3 = tf.Variable(tf.random_normal([256, 128]))
b3 = tf.Variable(tf.random_normal([128]))
sum3 = tf.add(tf.matmul(hidden_output2, W3), b3)
hidden_output3 = tf.nn.relu(sum3)

# 4nd Hidden Layer
W4 = tf.Variable(tf.random_normal([128, 128]))
b4 = tf.Variable(tf.random_normal([128]))
sum4 = tf.add(tf.matmul(hidden_output3, W4), b4)
hidden_output4 = tf.nn.relu(sum4)

# Output Layer
Wo = tf.Variable(tf.random_normal([128, 10]))
bo = tf.Variable(tf.random_normal([10]))
logits = tf.add(tf.matmul(hidden_output4, Wo), bo)
Y_prob = tf.nn.sigmoid(logits)

# Classification & Accuracy
n_of_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(n_of_correct, tf.float32))

# Cost(loss) function & Optimizer
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


### 머신이 학습해야 하는 파라미터(weight와  bias)의 갯수가 얼마나 되는지 계산해 봅시다. 

### 03. 세션을 만들고, 학습시키자!

In [0]:
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

training_epochs = 20
batch_size = 128
# 배치의 개수 미리 준비
n_of_batches = int(mnist.train.num_examples / batch_size)

for i in range(training_epochs + 1):
    print("{}th epoch".format(i))

    for batch in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch]})
        
        if (batch % 25 == 0):
            tr_a, loss = sess.run([accuracy, cost], feed_dict={X: X_batch, Y: np.eye(10)[Y_batch]})
            print("training loss: {:.4f} accuracy: {:.2f}%".format(loss, tr_a * 100))

print("Training Complete")
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: np.eye(10)[mnist.test.labels]})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))
# print(sess.run([W[:, 0], b]))
# sess.close()

### 성능을 한 번 확인해 보자. 제대로 되고 있는걸까?

In [0]:
n = int(np.random.uniform(0, 9999, 1))
Predicted_distribution = sess.run(Y_prob, feed_dict={X: [mnist.test.images[n]]})
Predicted_distribution = Predicted_distribution[0]
pd_dict = {i: '%.2f' % prob for i, prob in enumerate(Predicted_distribution)}

print(n)
print(pd_dict)
mnist_plot(n)


### 좀 더 멋지게
1. hyper parameter 중 - 변수 초기화 알고리즘 사용
2. 그래프를 보기에 더 좋게 하려면 => name_scope

In [0]:
# Graph Clear & Make your Graph reproducible
tf.reset_default_graph()
tf.set_random_seed(2017)

# Hyper-Parameters & Option
learning_rate = 0.01
training_epochs = 20
batch_size = 128

# Input Layer, Real Y
with tf.name_scope("Input_Layer"):
    X = tf.placeholder(dtype=tf.float32, shape=[None, 28 * 28])
with tf.name_scope("Labels"):
    Y = tf.placeholder(dtype=tf.float32, shape=[None, 10])

# 1st Hidden Layer
with tf.name_scope("Hidden_Layer_1"):
    W1 = tf.get_variable(shape=[28 * 28, 512], initializer=tf.contrib.layers.xavier_initializer(), name="W1")
    b1 = tf.Variable(tf.truncated_normal([512], mean=0, stddev=0.5))
    sum1 = tf.add(tf.matmul(X, W1), b1)
    hidden_output1 = tf.nn.relu(sum1)

# 2nd Hidden Layer
with tf.name_scope("Hidden_Layer_2"):
    W2 = tf.get_variable(shape=[512, 512], initializer=tf.contrib.layers.xavier_initializer(), name="W2")
    b2 = tf.Variable(tf.truncated_normal([512], mean=0, stddev=0.5))
    sum2 = tf.add(tf.matmul(hidden_output1, W2), b2)
    hidden_output2 = tf.nn.relu(sum2)

# 3rd Hidden Layer
with tf.name_scope("Hidden_Layer_3"):
    W3 = tf.get_variable(shape=[512, 256], initializer=tf.contrib.layers.xavier_initializer(), name="W3")
    b3 = tf.Variable(tf.truncated_normal([256], mean=0, stddev=0.5))
    sum3 = tf.add(tf.matmul(hidden_output2, W3), b3)
    hidden_output3 = tf.nn.relu(sum3)
    
# 4nd Hidden Layer
with tf.name_scope("Hidden_Layer_4"):
    W4 = tf.get_variable(shape=[256, 128], initializer=tf.contrib.layers.xavier_initializer(), name="W4")
    b4 = tf.Variable(tf.truncated_normal([128], mean=0, stddev=0.5))
    sum4 = tf.add(tf.matmul(hidden_output3, W4), b4)
    hidden_output4 = tf.nn.relu(sum4)

# Output Layer
with tf.name_scope("Output_Layer"):
    Wo = tf.get_variable(shape=[128, 10], initializer=tf.contrib.layers.xavier_initializer(), name="Wo")
    bo = tf.Variable(tf.truncated_normal([10], mean=0, stddev=0.5))
    logits = tf.add(tf.matmul(hidden_output4, Wo), bo)
    Y_prob = tf.nn.softmax(logits)

# Accuracy and cost
with tf.name_scope("Accuracy_and_Cost"):
    n_of_correct = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
    accuracy = tf.reduce_mean(tf.cast(n_of_correct, tf.float32))
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y))

# Optimizer
with tf.name_scope("Optimizer"):
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)


In [0]:
# 세션 초기화
sess = tf.Session()
tf.set_random_seed(2017)
sess.run(tf.global_variables_initializer())

# epochs, batch
training_epochs = 20
batch_size = 512

# 학습 iteration
for i in range(training_epochs + 1):
    print("{}th epoch".format(i))

    # 배치의 개수 미리 준비
    n_of_batches = int(mnist.train.num_examples / batch_size)
    
    for batch in range(n_of_batches):
        X_batch, Y_batch = mnist.train.next_batch(batch_size)
        # 학습 진행
        sess.run(optimizer, feed_dict={X: X_batch, Y: np.eye(10)[Y_batch]})
        
        # cost 상황 출력
        if (batch % 25 == 0):
            tr_a, loss = sess.run([accuracy, cost], feed_dict={X: X_batch, Y: np.eye(10)[Y_batch]})
            print("training loss: {:.4f} accuracy: {:.2f}%".format(loss, tr_a * 100))
            
print("Training Complete")        

# accuracy 출력
test_accuracy = sess.run(accuracy, feed_dict={X: mnist.test.images, Y: np.eye(10)[mnist.test.labels]})
print("Test Accuracy: {:.2f}%".format(test_accuracy * 100))

# print(sess.run([W[:, 0], b]))
sess.close()

## show_graph 함수 생성 코드 
아래 교육에서 그래프를 그려보기 위해서 필요한 함수입니다. <br>
교육과정을 위해 필요한 코드일 뿐 교육 내용에 본 코드가 포함은 되지 않으니 이해하지 못하셔도 상관이 없습니다. 

In [0]:
from IPython.display import clear_output, Image, display, HTML

"""
show_graph(tf.get_default_graph().as_graph_def())
"""

def strip_consts(graph_def, max_const_size=32):
    """Strip large constant values from graph_def."""
    strip_def = tf.GraphDef()
    for n0 in graph_def.node:
        n = strip_def.node.add() 
        n.MergeFrom(n0)
        if n.op == 'Const':
            tensor = n.attr['value'].tensor
            size = len(tensor.tensor_content)
            if size > max_const_size:
                tensor.tensor_content = "<stripped %d bytes>"%size
    return strip_def

def show_graph(graph_def, max_const_size=32):
    """Visualize TensorFlow graph."""
    if hasattr(graph_def, 'as_graph_def'):
        graph_def = graph_def.as_graph_def()
    strip_def = strip_consts(graph_def, max_const_size=max_const_size)
    code = """
        <script>
          function load() {{
            document.getElementById("{id}").pbtxt = {data};
          }}
        </script>
        <link rel="import" href="https://tensorboard.appspot.com/tf-graph-basic.build.html" onload=load()>
        <div style="height:600px">
          <tf-graph-basic id="{id}"></tf-graph-basic>
        </div>
    """.format(data=repr(str(strip_def)), id='graph'+str(np.random.rand()))

    iframe = """
        <iframe seamless style="width:1000px;height:620px;border:0" srcdoc="{}"></iframe>
    """.format(code.replace('"', '&quot;'))
    display(HTML(iframe))

In [0]:
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
tf.reset_default_graph()
%reset

# (CHALLENGE!) MNIST is too easy, huh?!

## Here Comes the 'FASHION MNIST'



### 00. 라이브러리 불러오기

In [0]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

**Fashion MNIST Label**

*   0 - T-shirt/top
*   1 -	Trouser
*   2 -	Pullover
*   3 -	Dress
*   4 -	Coat
*   5 -	Sandal
*   6 -	Shirt
*   7 -	Sneaker
*   8 -	Bag
*   9 -	Ankle boot


### 01.  모든 전처리들
tf.keras.datasets.fashion_mnist.load_data() 함수 확인

In [0]:
'''
트레이닝 데이터로 활용할 60000개 이미지, 테스트용 10000개 이미지
28*28 사이즈의 이미지가 그냥 주욱 784칸 짜리 어레이에 담겨있다.
test, traing 데이터가 분리되어 있다.
레이블은 one-hot encoding을 해주어야 한다. 
'''

(train_x, train_y), (test_x, test_y) = tf.keras.datasets.fashion_mnist.load_data()

print(train_x.shape)
print(train_y.shape)
print(test_x.shape)
print(test_y.shape)

In [0]:
'''
Fashion mnist 시각화 함수
'''
def fashion_plot(i):
    pixels = test_x[i].reshape((28, 28))
    a = test_y.tolist()
    plt.title('Label : {}'.format(a[i]))
    plt.imshow(pixels, cmap='gray')
    return plt.show()

In [0]:
'''
반복실행해보자
'''
fashion_plot(np.random.randint(0, high=9999))

In [0]:
######################################################
#                    코딩하시면 됩니다!
######################################################

# Graph Clear & Make your Graph reproducible


# Hyper-Parameters & Option


# Input Layer, Real Y


# 1nd Hidden Layer


# 2nd Hidden Layer


# ... 원하는 만큼의 hidden layer를 구성해 보세요. 


# Output Layer


# Accuracy and cost


# Optimizer




In [0]:
######################################################
#                    코딩하시면 됩니다!
######################################################

# 세션 초기화

# epochs, batch

# 학습 iteration

# accuracy 출력

# 세션 초기화

# epochs, batch

# 배치의 개수 미리 준비

# 학습 iteration

# accuracy 출력

# print(sess.run([W[:, 0], b]))


In [0]:
show_graph(tf.get_default_graph().as_graph_def())

In [0]:
n = int(np.random.uniform(0, 9999, 1))
Predicted_distribution = sess.run(Y_prob, feed_dict={X: [test_x.reshape(-1, 784)[n]]})
Predicted_distribution = Predicted_distribution[0]

labels_list = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat', 
               'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle boot']

print(n)
for i, prob in enumerate(Predicted_distribution):
    print('%d. %s: %.2f' % (i, labels_list[i], prob))
fashion_plot(n)


# 성능이 만족스럽게 잘 나오나요? 

In [0]:
# W의 갯수: 
# B의 갯수: